In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
data=pd.read_csv('Videos_data.csv')
data.head()

,Unnamed: 0,Video Id,Title,Description,Category
0,0,iQeeqEeAK08,How I Became a TRAVEL VLOGGER | Make Money Eve...,"In this video, I share with you my tips on how...",travel
1,1,i9E_Blai8vk,Welcome to Bali | Travel Vlog | Priscilla Lee,DISCLAIMER* Please do not ride elephants when ...,travel
2,2,gtm2O_6_mUM,SIKKIM TRAVEL VLOG | Bagdogra to Gangtok - Vlo...,Hey everyone! I recently went on a trip to Sik...,travel
3,3,v6eh9zOqDnQ,Top 10 Travel Vloggers,"For this list, we're taking a look at YouTuber...",travel
4,4,BbegGNcvMb0,Travelling to Italy after Lockdown | Dhruv Rat...,Welcome to my First Vlog. Here I share with yo...,travel


In [3]:
data=data.drop('Unnamed: 0',axis=1)

In [4]:
data['Description']=data['Description'].fillna(' ')

In [5]:
data=data.drop(index=data[data.duplicated(subset='Title')].index)

In [6]:
# Change to lowercase
data['Title'] = data['Title'].map(lambda x: x.lower())
data['Description'] = data['Description'].map(lambda x: x.lower())

# Remove numbers
data['Title'] = data['Title'].map(lambda x: re.sub(r'\d+', '', x))
data['Description'] = data['Description'].map(lambda x: re.sub(r'\d+', '', x))

# Remove Punctuation
data['Title']  = data['Title'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))
data['Description']  = data['Description'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# Remove white spaces
data['Title'] = data['Title'].map(lambda x: x.strip())
data['Description'] = data['Description'].map(lambda x: x.strip())

# Tokenize into words
data['Title'] = data['Title'].map(lambda x: word_tokenize(x))
data['Description'] = data['Description'].map(lambda x: word_tokenize(x))
 
# Remove non alphabetic tokens
data['Title'] = data['Title'].map(lambda x: [word for word in x if word.isalpha()])
data['Description'] = data['Description'].map(lambda x: [word for word in x if word.isalpha()])

# filter out stop words
stop_words = set(stopwords.words('english'))
data['Title'] = data['Title'].map(lambda x: [w for w in x if not w in stop_words])
data['Description'] = data['Description'].map(lambda x: [w for w in x if not w in stop_words])

# Word Lemmatization
lem = WordNetLemmatizer()
data['Title'] = data['Title'].map(lambda x: [lem.lemmatize(word,"v") for word in x])
data['Description'] = data['Description'].map(lambda x: [lem.lemmatize(word,"v") for word in x])

# Turn lists back to string
data['Title'] = data['Title'].map(lambda x: ' '.join(x))
data['Description'] = data['Description'].map(lambda x: ' '.join(x))

In [8]:
data.to_csv('Finaldata.csv',index=False)

In [7]:
data['Text']=data['Title']+data['Description']

In [8]:
data.head()

,Video Id,Title,Description,Category,Text
0,iQeeqEeAK08,become travel vlogger make money even youre st...,video share tip become travel blogger make mon...,travel,become travel vlogger make money even youre st...
1,i9E_Blai8vk,welcome bali travel vlog priscilla lee,disclaimer please ride elephants visit country...,travel,welcome bali travel vlog priscilla leedisclaim...
2,gtm2O_6_mUM,sikkim travel vlog bagdogra gangtok vlog north...,hey everyone recently go trip sikkim friend sh...,travel,sikkim travel vlog bagdogra gangtok vlog north...
3,v6eh9zOqDnQ,top travel vloggers,list take look youtubers travel globe make us ...,travel,top travel vloggerslist take look youtubers tr...
4,BbegGNcvMb0,travel italy lockdown dhruv rathee vlog,welcome first vlog share journey italy lockdow...,travel,travel italy lockdown dhruv rathee vlogwelcome...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [10]:
cv = CountVectorizer()
count_matrix=cv.fit_transform(data['Text'])
similarity = cosine_similarity(count_matrix)
similarity

array([[1.        , 0.07755667, 0.05216405, ..., 0.        , 0.0541332 ,
        0.        ],
       [0.07755667, 1.        , 0.10619885, ..., 0.        , 0.        ,
        0.        ],
       [0.05216405, 0.10619885, 1.        , ..., 0.        , 0.02470831,
        0.03086067],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.33796318,
        0.06030227],
       [0.0541332 , 0.        , 0.02470831, ..., 0.33796318, 1.        ,
        0.09607689],
       [0.        , 0.        , 0.03086067, ..., 0.06030227, 0.09607689,
        1.        ]])

In [11]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix=tf.fit_transform(data['Text'])

In [12]:
titles=data['Title']
indices=pd.Series(data.index,index=data['Title'])

In [13]:
indices

Title
become travel vlogger make money even youre start youtuber                                0
welcome bali travel vlog priscilla lee                                                    1
sikkim travel vlog bagdogra gangtok vlog north east india kritika goel                    2
top travel vloggers                                                                       3
travel italy lockdown dhruv rathee vlog                                                   4
                                                                                       ... 
white paint green bad bad bad                                                          3463
fantasy fest body paint zombie bike ride duval street key west                         3464
nude girls body paint                                                                  3465
se abstract art action body paint untitled majorca falcon tower gd film bmpcc k uhd    3466
want artist paint portrait hirugao love affairs afternoon                 

In [33]:
def get_recom_video(title):
    idx=indices[title]
    scores=list(enumerate(similarity[idx]))
    scores=sorted(scores,key=lambda x:x[1],reverse=True)
    scores=scores[1:11]
    video_index=[i[0] for i in scores]
    videos_detail=list(data['Video Id'].iloc[video_index])
    return videos_detail

In [34]:
get_recom_video('top travel vloggers')

['ytP5Lpfjiik',
 'Yq8mBlb1L2I',
 'U95vNX_aVbA',
 'h_BZjMiOk_s',
 'uzJXE34UwHU',
 'cSw50Jw0H34',
 'G7kqBZs01o4',
 'NkRKL8IsbnY',
 'Y9cXTmraOVE',
 'fesetFJ6fLU']

In [33]:
get_recom_video('delicious food')

1392            czn burak cook delicious food short short
1616            czn burak cook delicious food short short
1638            czn burak cook delicious food short short
1654            czn burak cook delicious food short short
1611              czn burak cook delicious food food time
1273    eat palak chicken curry basmati rice delicious...
1286    satisfy video home cook show cook delicious fo...
243                   indian travel nubian villages sudan
1549    grill fish green mango lunch eat delicious foo...
1344    primitive technology eat delicious cook egg du...
Name: Title, dtype: object